In [ ]:
from pysheds.grid import Grid
from skimage.morphology import skeletonize
import matplotlib.pyplot as plt
from skimage.io import imread
import random
import numpy as np
import cv2
from PIL import Image
import os
import h5py

In [ ]:
# Parameters to change !!!

sketch_rivers_flag = True
sketch_basins_flag = True
sketch_ridges_flag = True
sketch_peaks_flag = True

imageDimensionX = 512
imageDimensionY = 512

# last tile nr
iterateFrom = 0

colorRivers = (255, 0, 0)
colorRidges = (0, 0, 255)
colorPeaks = (0, 0, 0)
colorBasins = (0, 255, 0)

savePathDestination = 'images/trainGIS/terrain_'
readDEMPath = 'heightmapsDEM/topography_'
readTIFFPath = 'heightmapsTIFF/topography_'

In [ ]:
def sketch_rivers(tiff_image):
    grid = Grid.from_raster(tiff_image, data_name='dem')
    grid.fill_depressions(data='dem', out_name='flooded_dem')
    depressions =  grid.detect_depressions('flooded_dem')
    grid.resolve_flats(data='flooded_dem', out_name='inflated_dem')

    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
    grid.accumulation(data='dir', out_name='acc', dirmap=dirmap)
    
    upsampled_rivers = np.log(grid.view('acc') + 1)
    upsampled_rivers = (upsampled_rivers - np.amin(upsampled_rivers)) / (np.amax(upsampled_rivers) - np.amin(upsampled_rivers))
    upsampled_rivers = np.array(upsampled_rivers * 255, dtype=np.uint8)
    _, thresholded_river = cv2.threshold(upsampled_rivers, 170, 255, cv2.THRESH_BINARY)
    thresholded_river[thresholded_river == 255] = 1
    skeletonized_rivers = skeletonize(thresholded_river)

    rivers_sketch = Image.fromarray(skeletonized_rivers)
    rgb_rivers_sketch = rivers_sketch.convert('RGB')
    
    basins_sketch = Image.fromarray(depressions)
    rgb_basins_sketch = basins_sketch.convert('RGB')
    
    return rgb_rivers_sketch, rgb_basins_sketch

In [ ]:
def sketch_ridges(tiff_image):
    grid = Grid.from_raster(tiff_image, data_name='dem')
    grid.dem = grid.dem.max() - grid.dem
    grid.fill_depressions(data='dem', out_name='flooded_dem')
    peaks = grid.detect_depressions('flooded_dem')
    
    grid.resolve_flats(data='flooded_dem', out_name='inflated_dem')
    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
    grid.accumulation(data='dir', out_name='acc', dirmap=dirmap)

    upsampled_ridges = np.log(grid.view('acc') + 1)
    upsampled_ridges = (upsampled_ridges - np.amin(upsampled_ridges)) / (np.amax(upsampled_ridges) - np.amin(upsampled_ridges))
    upsampled_ridges = np.array(upsampled_ridges * 255, dtype=np.uint8)
    _, thresholded_ridges = cv2.threshold(upsampled_ridges, 170, 255, cv2.THRESH_BINARY)
    thresholded_ridges[thresholded_ridges == 255] = 1
    skeletonized_ridges = skeletonize(thresholded_ridges)
    ridges_sketch = Image.fromarray(skeletonized_ridges)
    rgb_ridges_sketch = ridges_sketch.convert('RGB')
    
    peaks_sketch = Image.fromarray(peaks)
    rgb_peaks_sketch = peaks_sketch.convert('RGB')
    
    return rgb_ridges_sketch, rgb_peaks_sketch

In [ ]:
def prepare_heightmap_sketch(file_path):
    rivers, basins = sketch_rivers(file_path)
    ridges, peaks = sketch_ridges(file_path)

    sketch = np.zeros([imageDimensionX,imageDimensionY,3],dtype=np.uint8)
    sketch.fill(255)
        
    if sketch_rivers_flag == True:
        pixels_rivers = rivers.load()
  
        for i in range(rivers.size[0]):
            for j in range(rivers.size[1]):
                if pixels_rivers[i,j] == (255, 255, 255):
                    sketch[j,i] = colorRivers
    
    
    if sketch_ridges_flag == True:
        pixels_ridges = ridges.load()
            
        for i in range(ridges.size[0]):
            for j in range(ridges.size[1]):
                if pixels_ridges[i,j] == (255, 255, 255):
                    sketch[j,i] = colorRidges 
    
    
    if sketch_peaks_flag == True:
        pixels_peaks = peaks.load()

        for i in range(peaks.size[0]):
            for j in range(peaks.size[1]):
                if pixels_peaks[i,j] == (255, 255, 255):
                    sketch[j,i] = colorPeaks  
            
    
    if sketch_basins_flag == True:
        pixels_basins = basins.load()
            
        for i in range(basins.size[0]):
            for j in range(basins.size[1]):
                if pixels_basins[i,j] == (255, 255, 255):
                    sketch[j,i] = colorBasins

    
    return sketch

In [ ]:
files_len = len(os.listdir('heightmapsDEM')) * 4
files_len_directory = len(os.listdir('heightmapsDEM'))

idxs = [x for x in range(files_len)]
np.random.shuffle(idxs)

In [ ]:
dataset_iterator = iterateFrom
iterator = 0

for iterator in range(files_len_directory):       
    print(str(dataset_iterator) + '/' + str(files_len + iterateFrom))
    sketch = prepare_heightmap_sketch(readDEMPath + str(iterator+1) + '.tif')
    heightmap = cv2.imread(readTIFFPath + str(iterator+1) + '.tif')
    
    print('topography_' + str(iterator+1) + '.tif')

    sketch_array = np.array(sketch)
    heightmap_array = np.array(heightmap)
    
    im_h = cv2.hconcat([sketch_array, heightmap_array])
    cv2.imwrite(savePathDestination + str(idxs[dataset_iterator]) + ".png",im_h)
    dataset_iterator += 1
        
    # rotate90
    sketch_array = np.rot90(sketch_array)
    heightmap_array = np.rot90(heightmap_array)
        
    im_h = cv2.hconcat([sketch_array, heightmap_array])
    cv2.imwrite(savePathDestination + str(idxs[dataset_iterator]) + ".png",im_h)
    dataset_iterator += 1
        
    # rotate90
    sketch_array = np.rot90(sketch_array)
    heightmap_array = np.rot90(heightmap_array)
        
    im_h = cv2.hconcat([sketch_array, heightmap_array])
    cv2.imwrite(savePathDestination + str(idxs[dataset_iterator]) + ".png",im_h)    
    dataset_iterator += 1
        
    # rotate90
    sketch_array = np.rot90(sketch_array)
    heightmap_array = np.rot90(heightmap_array)
    
    im_h = cv2.hconcat([sketch_array, heightmap_array])
    cv2.imwrite(savePathDestination + str(idxs[dataset_iterator]) + ".png",im_h)
    dataset_iterator += 1

print(str(dataset_iterator) + '/' + str(files_len + iterateFrom))
print("Done!")

In [ ]:
#f.close()